# Анна Запорощенко

На последнем семинаре мы проанализировали несколько различных морфологических теггеров. Как же узнать, какой использовать? Давайте сравним их качество!
В этой домашке вам будет нужно найти или самим написать русский и английский тексты (каждый от ста слов), в которых  будут какие-то трудные или неоднозначные для POS теггинга моменты и разметить их в ручную (а потом объяснить, какие моменты вы тут считаете трудными для автоматического посттеггинга и почему) – с помощью этих текстов мы будем оценивать качество работы наших теггоров. В текстах размечаем только части речи, ничего больше!
Вы получаете балл за создание, разметку текста и объяснение того, почему этот текст подходит для оценки (что в нём сложного). Всего за этот пункт 2 балла, т. к. языка 2.
Потом вам будет нужно взять три  POS теггера для русского (pymorphy2, mysteam, Natasha) и 3  - для английского (SpyCy, Flair, NLTK) и «прогнать» текст через каждый из них (если вы запустите только 2 теггера из трёх – получите балл, если три  из трёх – 2 балла, т. е. суммарно за этот пункт можно получить 4 балла).
После этого вам надо будет оценить accuracy для каждого теггера. Заметьте, что в разных системах имена теггов и части речи  могут отличаться, – вам надо будет свести это всё к единому стандарту с помощью какой-то функции или кода и сравнить с вашим размеченным руками эталоном - тоже с помощью какого-то кода или функции. Этот пункт стоит 2 балла.
Тут вы уже получили 8 баллов.
Дальше вам нужно взять лучший теггер для русского языка и  с его помощью написать функцию, которая повысит качество работы программы из первой домашки. Так, многие из вас справедливо заметили, что если бы мы могли класть в словарь не только отдельные слова, но и словосочетания, то программа работала бы лучше. Вам надо выделить 3 вида синтаксических групп (к примеру не + какая-то часть речи или NP или сущ.+ наречие или еще что-то), запись которых в словарь, по вашему мнению, улучшила бы качество работы программы и создать такую функцию или функции, которые с помощью любых известных нам средств (chunking и regexp grammar, Natasha syntax parser,  код с последнего занятия по SpyCy, etc.) будет выделять эти группы в поданном в нее тексте. Два балла за саму функцию, балл за объяснение того, почему именно эти группы вы взяли.
2 бонусных балла, если встроите эту функцию в программу из предыдущей домашки и сравните качества работы программы с нею и без неё.

In [121]:
import nltk
from nltk import word_tokenize
from nltk import sent_tokenize
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
import re
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
import spacy
from flair.data import Sentence
from flair.models import SequenceTagger
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\VADIK\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\VADIK\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [122]:
def cleaner(text):
    text = word_tokenize(text)
    text_clean =[]
    for word in text:
        if (word not in '.,/\|!?:;*—- '):
            text_clean.append(word)
    return text_clean

In [123]:
text_rus = 'Германия, Дания и Нидерланды намерены заключить соглашение о строительстве в Северном море искусственного острова, \
на котором будет производиться электроэнергия для снабжения электричеством 80 миллионов человек. По проекту на острове под \
названием Power Link площадью около 6 км² оборудуют порт, взлетно-посадочную полосу, жилые и технические сооружения. \
Но самое главное — остров будет окружен тысячами ветровых турбин, предназначенных для производства экологически чистой \
энергии. Остров построят на Доггер-банке — большой песчаной отмели в Северном море, примерно на равном расстоянии от \
Великобритании, Скандинавии, Германии и стран Бенилюкса. Он будет соединен с этими территориями подводными кабелями. \
Глубина моря в районе отмели колеблется от 15 до 36 метров, что позволит без особых сложностей установить 10 тысяч ветровых \
турбин. Кроме того, на Доггер-банке всегда очень ветрено — этот факт также повлиял на выбор места для возведения острова. \
Остров станет своего рода распределительным центром, через который энергия от ветровых электростанций будет подаваться в \
страны ЕС: в Нидерланды, Данию, Германию и др. Автор статьи: Мария Смирных.'

In [157]:
pos_rus = ['NOUN', 'NOUN', 'CONJ', 'NOUN', 'ADJ', 'VERB', 'NOUN', 'PREP', 'NOUN', 'PREP', 'ADJ', 
       'NOUN', 'ADJ', 'NOUN', 'PREP', 'PRO', 'VERB', 'VERB', 'NOUN', 'PRO', 'NOUN', 'NOUN',
       'NUMR', 'NUMR', 'NOUN',
       'PREP', 'NOUN', 'PREP', 'NOUN', 'PREP', 'NOUN', 'OTHER', 'OTHER', 'NOUN', 'PREP', 'NUMR',
        'NOUN', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'ADJ', 'CONJ', 'ADJ', 'NOUN',
       'CONJ', 'PRO', 'NOUN', 'NOUN', 'VERB', 'VERB', 'NUMR', 'ADJ', 'NOUN', 'VERB', 'PREP', 'NOUN', 
        'ADVB', 'ADJ', 'NOUN',
       'NOUN', 'VERB', 'PREP', 'NOUN', 'ADJ', 'ADJ', 'NOUN', 'PREP', 'ADJ', 'NOUN', 'ADVB', 'PREP',
      'ADJ', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'NOUN',
       'PRO', 'VERB', 'VERB', 'PREP', 'PRO', 'NOUN', 'ADJ', 'NOUN',
       'NOUN', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'VERB', 'PREP', 'NUMR', 'PREP', 'NUMR', 'NOUN', 'CONJ', 'VERB', 
       'PREP', 'ADJ', 'NOUN', 'VERB', 'NUMR', 'NUMR', 'ADJ', 'NOUN',
       'PREP', 'PRO', 'PREP', 'NOUN', 'ADVB', 'ADVB', 'ADVB', 'PRO', 'NOUN', 'CONJ', 'VERB', 'PREP',
      'NOUN', 'NOUN', 'PREP', 'NOUN', 'NOUN',
       'NOUN', 'VERB', 'PRO', 'NOUN', 'ADJ', 'NOUN', 'PREP', 'PRO', 'NOUN', 'PREP', 'ADJ', 'NOUN', 
       'VERB', 'VERB', 'PREP', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'CONJ', 'PRO',
       'NOUN', 'NOUN', 'NOUN', 'NOUN']

In [158]:
len(pos_rus)

154

Проблемы для русского языка:
* Слова с дефисом, неразделимые: Доггер-банк
* Фамилии, похожие на прилагательные: Смирных
* Аббревиатуры: ЕС
* Сокращения: др., км²
* Частеричные категории: мест./прил.(который, самый), сущ./числ. (миллионов), предл./нар. (около), сущ./прил. (главное), прил./гл. (предназначенных)
* Омонимия: сущ./гл. (отмели)
* Редкие слова: Бинелюкса

In [159]:
tokens_rus = cleaner(text_rus)

In [160]:
pos_rus_pymorphy = []
morph0 = MorphAnalyzer()
for p in tokens_rus:
    p = morph0.parse(p)[0]
    p = str(p.tag.POS)
    if (p == 'PRTS')or(p == 'INFN')or(p == 'PRTF')or(p == 'GRND'):
        p = 'VERB'
    if (p == 'ADJF')or(p == 'ADJS'):
        p = 'ADJ'
    if p == 'None':
        p = 'OTHER'
    if p == 'NPRO':
        p = 'PRO'
    pos_rus_pymorphy.append(p)

In [161]:
pos_rus_pymorphy

['NOUN',
 'NOUN',
 'CONJ',
 'NOUN',
 'VERB',
 'VERB',
 'NOUN',
 'PREP',
 'NOUN',
 'PREP',
 'ADJ',
 'NOUN',
 'ADJ',
 'NOUN',
 'PREP',
 'ADJ',
 'VERB',
 'VERB',
 'NOUN',
 'PREP',
 'NOUN',
 'NOUN',
 'OTHER',
 'NOUN',
 'NOUN',
 'PREP',
 'NOUN',
 'PREP',
 'NOUN',
 'PREP',
 'NOUN',
 'OTHER',
 'OTHER',
 'NOUN',
 'PREP',
 'OTHER',
 'OTHER',
 'VERB',
 'NOUN',
 'ADJ',
 'NOUN',
 'ADJ',
 'CONJ',
 'ADJ',
 'NOUN',
 'CONJ',
 'ADJ',
 'ADJ',
 'NOUN',
 'VERB',
 'ADJ',
 'NOUN',
 'ADJ',
 'NOUN',
 'VERB',
 'PREP',
 'NOUN',
 'ADVB',
 'ADJ',
 'NOUN',
 'NOUN',
 'VERB',
 'PREP',
 'NOUN',
 'ADJ',
 'ADJ',
 'NOUN',
 'PREP',
 'ADJ',
 'NOUN',
 'ADVB',
 'PREP',
 'ADJ',
 'NOUN',
 'PREP',
 'NOUN',
 'NOUN',
 'NOUN',
 'CONJ',
 'NOUN',
 'NOUN',
 'PRO',
 'VERB',
 'VERB',
 'PREP',
 'ADJ',
 'NOUN',
 'ADJ',
 'NOUN',
 'NOUN',
 'NOUN',
 'PREP',
 'NOUN',
 'NOUN',
 'VERB',
 'PREP',
 'OTHER',
 'PREP',
 'OTHER',
 'NOUN',
 'CONJ',
 'VERB',
 'PREP',
 'ADJ',
 'NOUN',
 'VERB',
 'OTHER',
 'NOUN',
 'ADJ',
 'NOUN',
 'PREP',
 'ADJ',
 'PRE

In [162]:
accuracy_score(pos_rus_pymorphy, pos_rus)

0.8441558441558441

In [163]:
morph1 = Mystem()
pos_rus_mystem = morph1.analyze((' ').join(tokens_rus))[:-1]
val = {'text': ' '}
while val in pos_rus_mystem:
    pos_rus_mystem.remove(val)
for pos in enumerate(pos_rus_mystem):
    if ('analysis' in pos[1])and(len(pos[1]['analysis']) != 0):
        pos_rus_mystem[pos[0]] = re.split(',|=', pos[1]['analysis'][0]['gr'])[0]
    elif pos[1]['text'] == '-':
        pos_rus_mystem[pos[0]] = 'PUNKT'
    else:
        pos_rus_mystem[pos[0]] = 'OTHER'

In [164]:
for pos in enumerate(pos_rus_mystem):
    if pos[1] == 'S':
        pos_rus_mystem[pos[0]] = 'NOUN'
    if pos[1] == 'A':
        pos_rus_mystem[pos[0]] = 'ADJ'
    if pos[1] == 'PR':
        pos_rus_mystem[pos[0]] = 'PREP'
    if (pos[1] == 'APRO')or(pos[1] == 'ADVPRO')or(pos[1] == 'SPRO'):
        pos_rus_mystem[pos[0]] = 'PRO'
    if pos[1] == 'V':
        pos_rus_mystem[pos[0]] = 'VERB'
    if pos[1] == 'ADV':
        pos_rus_mystem[pos[0]] = 'ADVB'

In [165]:
pos_rus_mystem = re.sub(' [A-Z]+? PUNKT [A-Z]+? ', ' FALSE ', ' '.join(pos_rus_mystem)).split(' ')

In [166]:
len(pos_rus_mystem)

154

In [167]:
accuracy_score(pos_rus_mystem, pos_rus)

0.8961038961038961

In [168]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [169]:
doc = Doc((' ').join(tokens_rus))
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
pos_rus_natasha = []
for token in doc.tokens:
    pos = re.findall("pos='(.+?)'",str(token))[0]
    if pos == 'PROPN':
        pos = 'NOUN'
    if (pos == 'PRON')or(pos == 'DET'):
        pos = 'PRO'
    if pos == 'AUX':
        pos = 'VERB'
    if pos == 'ADP':
        pos = 'PREP'
    if pos == 'CCONJ':
        pos = 'CONJ'
    if pos == 'X':
        pos = 'OTHER'
    if pos == 'NUM':
        pos = 'NUMR'
    if pos == 'ADV':
        pos = 'ADVB'
    pos_rus_natasha.append(pos)

In [170]:
len(pos_rus_natasha)

154

In [171]:
accuracy_score(pos_rus_natasha, pos_rus)

0.935064935064935

In [195]:
text_en = "This condition is in contradictory to the large θc as is expected for sessile droplets on highly \
lyophobic surfaces, which makes it difficult to produce uniform OSC films by conventional solution processes. \
So, it's important and necessary to control the shape of solution meniscus on the lyophobic surfaces. Here, \
we report the development of an extended meniscus coating technique that allows to manufacture \
OSC single crystal films on top of highly lyophobic Cytop gate dielectric layer. The uniform film growth becomes \
possible by keeping extension of solution meniscus on the Cytop surface. The electromagnetic coating allowed us \
to integrate single crystal films of small molecular OSCs and Cytop gate dielectric in BGBC type \
organic TFTs, resulting in an extremely small SS value of 63 mV dec1, which is close to the theoretical limit."

In [196]:
pos_en = ['DET', 'NOUN', 'VERB', 'PREP', 'NOUN', 'PREP', 'DET', 'ADJ', 'OTHER', 'CONJ', 'VERB', 'VERB', 'PREP',
          'ADJ', 'NOUN', 'PREP', 'ADVB', 'ADJ', 'NOUN', 'PRO', 'VERB', 'PRO', 'ADJ', 'PREP', 'VERB', 'ADJ', 'NOUN',
          'NOUN', 'PREP', 'ADJ', 'NOUN', 'NOUN',
         'CONJ', 'PRO', 'VERB', 'ADJ', 'CONJ', 'ADJ', 'PREP', 'VERB', 'DET', 'NOUN', 'PREP', 'NOUN', 'NOUN', 
         'PREP', 'DET', 'ADJ', 'NOUN',
         'ADVB', 'PRO', 'VERB', 'DET', 'NOUN', 'PREP', 'DET', 'VERB', 'NOUN', 'VERB', 'NOUN', 'DET',
         'VERB', 'PREP', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'PREP', 'ADVB', 'ADJ', 'NOUN', 
         'NOUN', 'NOUN', 'NOUN',
         'DET', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'PREP', 'VERB', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'PREP', 'DET',
         'NOUN', 'NOUN',
         'DET', 'ADJ', 'NOUN', 'VERB', 'PRO', 'PREP', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'ADJ', 'ADJ',
         'NOUN', 'CONJ', 'NOUN', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'PREP', 'DET', 
         'ADVB', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'NUMR', 'OTHER', 'OTHER', 'PRO', 'VERB', 'ADJ', 'PREP', 'DET', 'ADJ',
         'NOUN']

In [197]:
tokens_en = cleaner(text_en)

In [198]:
len(pos_en)

134

In [199]:
len(tokens_en)

134

Проблемы для английского языка:
* Сокращения: it's
* Омонимия: as (conj/prep), uniform (adj/noun), so (advb/conj), manufacture(verb/noun)
* Герундий
* Аббревиатура: OSC, SS
* Специализированные слова, в т. ч. имена собственные: sessile, meniscus, Cytop

In [200]:
pos = nltk.pos_tag(tokens_en)
pos_en_nltk = []

In [201]:
for pos in pos_en:
    if pos[1] == 'DT':
        p = 'DET'
    if (pos[1] == 'NN')or(pos[1] == 'NNP')or(pos[1] == 'NNS'):
        p = 'NOUN'
    if (pos[1] == 'VBZ')or(pos[1] == 'VB'):
        p = 'VERB'
    if (pos[1] == 'TO')or(pos[1] == 'IN'):
        p = 'PREP'
    if pos[1] == 'JJ':
        p = 'ADJ'
    if pos[1] == 'CC':
        p = 'CONJ'
    if pos[1] == 'RB':
        p = 'ADVB'
    if (pos[1] == 'PRP')or(pos[1] == 'WDT'):
        p = 'PRO'
    if pos[1] == 'CD':
        p = 'NUMR'
    pos_en_nltk.append(p)

In [203]:
accuracy_score(pos_en_nltk, pos_en)

0.3208955223880597

In [204]:
#nltk.help.upenn_tagset('WDT')

In [205]:
nlp = spacy.load("en_core_web_sm")

In [206]:
pos_en_spacy = []
doc = nlp((' ').join(tokens_en))
for sent in doc.sents:
    for pos in sent:
        pos = pos.pos_
        if pos == 'PROPN':
            pos = 'NOUN'
        if (pos == 'PRON')or(pos == 'DET'):
            pos = 'PRO'
        if pos == 'AUX':
            pos = 'VERB'
        if pos == 'ADP':
            pos = 'PREP'
        if (pos == 'CCONJ')or(pos == 'SCONJ'):
            pos = 'CONJ'
        if pos == 'X':
            pos = 'OTHER'
        if pos == 'NUM':
            pos = 'NUMR'
        if pos == 'ADV':
            pos = 'ADVB'
        pos_en_spacy.append(pos)

In [209]:
len(pos_en_spacy)

134

In [207]:
accuracy_score(pos_en_spacy, pos_en)

0.8208955223880597

In [210]:
def cleaner_2(text):
    text = word_tokenize(text)
    text_clean =[]
    for word in text:
        if (word not in '.,/\|!?:;*—- ')and(word.startswith("'") == False):
            text_clean.append(word)
        elif word.startswith("'") == True:
            text_clean[-1] = text_clean[-1] + word
    return text_clean

In [211]:
sents_en = sent_tokenize(text_en)
for sent in enumerate(sents_en):
    sents_en[sent[0]] = (' ').join(cleaner_2(sent[1]))

In [212]:
pos_en_flair = []
for sent in sents_en:
    sentence = Sentence(sent)
    tagger = SequenceTagger.load('pos')
    tagger.predict(sentence)
    pos_s = re.findall(r'<.+?>', sentence.to_tagged_string())
    pos_en_flair.extend(pos_s)

2020-10-18 01:49:42,579 loading file C:\Users\VADIK\.flair\models\en-pos-ontonotes-v0.5.pt
2020-10-18 01:49:44,097 loading file C:\Users\VADIK\.flair\models\en-pos-ontonotes-v0.5.pt
2020-10-18 01:49:45,131 loading file C:\Users\VADIK\.flair\models\en-pos-ontonotes-v0.5.pt
2020-10-18 01:49:46,526 loading file C:\Users\VADIK\.flair\models\en-pos-ontonotes-v0.5.pt
2020-10-18 01:49:47,595 loading file C:\Users\VADIK\.flair\models\en-pos-ontonotes-v0.5.pt


In [213]:
for pos in enumerate(pos_en_flair):
    if pos[1] == '<DT>':
        pos_en_flair[pos[0]] = 'DET'
    if (pos[1] == '<NN>')or(pos[1] == '<NNP>')or(pos[1] == '<NNS>'):
        pos_en_flair[pos[0]] = 'NOUN'
    if (pos[1] == '<VBZ>')or(pos[1] == '<VB>')or(pos[1] == '<VBN>')or(pos[1] == '<VBP>'):
        pos_en_flair[pos[0]] = 'VERB'
    if (pos[1] == '<TO>')or(pos[1] == '<IN>'):
        pos_en_flair[pos[0]] = 'PREP'
    if pos[1] == '<JJ>':
        pos_en_flair[pos[0]] = 'ADJ'
    if pos[1] == '<CC>':
        pos_en_flair[pos[0]] = 'CONJ'
    if pos[1] == '<RB>':
        pos_en_flair[pos[0]] = 'ADVB'
    if (pos[1] == '<PRP>')or(pos[1] == '<WDT>'):
        pos_en_flair[pos[0]] = 'PRO'
    if pos[1] == '<CD>':
        pos_en_flair[pos[0]] = 'NUMR'
    if pos[1] == '<``>':
        pos_en_flair[pos[0]] = 'OTHER'

In [214]:
len(pos_en_flair)

134

In [215]:
accuracy_score(pos_en_flair, pos_en)

0.9104477611940298